In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px

2023-12-14 09:31:01.386 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
def get_data_from_csv():
    df = pd.read_csv('all_sales_data.csv')
    return df
df = get_data_from_csv()

### MASTER DATA ###
all_sales = df.copy()

In [3]:
all_sales['Invoice Date'] = pd.to_datetime(all_sales['Invoice Date'])
all_sales['Invoice Date'] = all_sales['Invoice Date'].dt.normalize()
all_sales['Invoice Date'] = all_sales['Invoice Date'].dt.floor('D')
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86985 entries, 0 to 86984
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Dollars                86985 non-null  float64       
 1   Customer               86985 non-null  object        
 2   Customer Order Number  86985 non-null  object        
 3   Dollars Formatted      86985 non-null  float64       
 4   Invoice Date           86985 non-null  datetime64[ns]
 5   Item Full Description  86985 non-null  object        
 6   Market Segment         86985 non-null  object        
 7   Parent Customer        86985 non-null  object        
 8   Sale Origin            86985 non-null  object        
 9   Sales by Parent        86985 non-null  float64       
 10  Suffix                 3990 non-null   object        
 11  Vistar Retail          86985 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(8)
memory usage: 8.

In [4]:
year = all_sales['Invoice Date'].dt.year.unique()
segment = all_sales['Market Segment'].unique()

In [5]:
df_selection = all_sales[
    (all_sales['Invoice Date'].dt.year.isin(year)) &
    (all_sales['Market Segment'].isin(segment))
    ]
df_selection

,Dollars,Customer,Customer Order Number,Dollars Formatted,Invoice Date,Item Full Description,Market Segment,Parent Customer,Sale Origin,Sales by Parent,Suffix,Vistar Retail
0,34.9875,Amazon SC,SO-00099997,34.987500,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2.473553e+06,NaN,No
1,34.9875,Amazon SC,SO-00099972,34.987500,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2.473553e+06,NaN,No
2,34.9875,Amazon SC,SO-00100026,34.987500,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2.473553e+06,NaN,No
3,34.9875,Amazon SC,SO-00100069,34.987500,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2.473553e+06,NaN,No
4,34.9875,Amazon SC,SO-00100085,34.987500,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2.473553e+06,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...
86980,2575.1250,Vistar/ VSA Corporate,SO-00074694,2.575125,2022-04-01,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1.978810e+06,K,Yes
86981,1383.4650,Vistar/ VSA Corporate,SO-00079448,1.383465,2022-07-18,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1.978810e+06,K,Yes
86982,122.6250,Vistar/ VSA Corporate,SO-00074694,122.625000,2022-04-01,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1.978810e+06,NaN,Yes
86983,1509.2325,Vistar/ VSA Corporate,SO-00079448,1.509232,2022-07-18,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1.978810e+06,K,Yes


In [67]:
year_u = df_selection['Invoice Date'].dt.year
week_u = df_selection['Invoice Date'].dt.isocalendar().week

sales_per_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Market Segment  812 non-null    object 
 1   Dollars         812 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.8+ KB


In [75]:
df_selection.groupby(['Invoice Date','Market Segment'])[['Dollars','Market Segment']].sum()

Dollars
Invoice Date Market Segment                      
2022-01-03   Alternate Retail             784.800
             Broadline Distributor        392.400
             Convenience                 2092.800
             Online                        26.265
             Other (to be categorized)     78.480
...                                           ...
2023-12-11   Broadline Distributor        282.000
             Convenience                  256.620
             Grocery                     3847.140
             Online                     20950.125
             Vending                     4743.240

[2880 rows x 1 columns]

In [72]:
# sales_per_day = df_selection.groupby(pd.Grouper(freq='W', key='Invoice Date'))[['Market Segment','Dollars']].sum()
sales_per_day = df_selection.groupby(['Market Segment',year,week],as_index=False)['Dollars'].transform(func='sum')
fig_sales_per_day = px.scatter(
    sales_per_day,
    x=sales_per_day.index,
    y='Dollars',
    title='<b>Weekly Sales</b>',
    template = 'plotly_white',
    labels={'Invoice Date':'',
            'Dollars':'<b>$USD</b>'},
    trendline="rolling", trendline_options=dict(function="mean", window=6), trendline_scope="overall",
    color = 'Market Segment'

)
fig_sales_per_day.show()

ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['Dollars'] but received: Market Segment

In [71]:
sales_per_day

,Dollars
0,38890.2850
1,38890.2850
2,38890.2850
3,38890.2850
4,38890.2850
...,...
86980,64479.4750
86981,58343.1775
86982,64479.4750
86983,58343.1775


In [8]:
df_selection = all_sales[(all_sales['Invoice Date'].dt.year.isin(year)) & (all_sales['Market Segment'].isin(segment))]

In [17]:
df_selection[['Invoice Date', 'Sale Origin', 'Market Segment', 'Parent Customer', 'Customer', 'Customer Order Number','Item Full Description','Dollars']].round(2).sort_values(by='Invoice Date').reset_index(drop=True)

,Invoice Date,Sale Origin,Market Segment,Parent Customer,Customer,Customer Order Number,Item Full Description,Dollars
0,2022-01-03,Unleashed,Online,Shopify,Shopify USA Mello,SO-00071293,Mello Chocolate - Dark Pouch 102g / 3.6oz,0.00
1,2022-01-03,Unleashed,Online,Shopify,Shopify USA Mello,SO-00071305,Mello Chocolate - Dark Pouch 102g / 3.6oz,0.00
2,2022-01-03,Unleashed,Online,Shopify,Shopify USA Mello,SO-00071314,Mello Chocolate - Milk Pouch 102g / 3.6oz,0.00
3,2022-01-03,Unleashed,Online,Shopify,Shopify USA Mello,SO-00071356,Mello Chocolate - Milk Pouch 102g / 3.6oz,0.00
4,2022-01-03,Dot,Alternate Retail,Tropical,Tropical Texas,2322411,CAFFEINATED CHOCOLATE BITES SINGLES DARK,261.60
...,...,...,...,...,...,...,...,...
86980,2023-12-11,Unleashed,Online,Amazon,Amazon SC,SO-00131429,50ct Pouch - Dark Choc Bites,34.99
86981,2023-12-11,Unleashed,Online,Amazon,Amazon SC,SO-00131422,50ct Pouch - Dark Choc Bites,34.99
86982,2023-12-11,Unleashed,Online,Amazon,Amazon SC,SO-00131420,50ct Pouch - Dark Choc Bites,34.99
86983,2023-12-11,Unleashed,Online,Amazon,Amazon SC,SO-00131460,50ct Pouch - Dark Choc Bites,34.99


In [19]:
df_selection['Invoice Date'] = pd.to_datetime(df_selection['Invoice Date'])
df_selection['Invoice Date'] = df_selection['Invoice Date'].dt.normalize()

df_selection['Invoice Date'] = df_selection['Invoice Date'].dt.floor('D')
df_selection.round(2)

,Dollars,Customer,Customer Order Number,Dollars Formatted,Invoice Date,Item Full Description,Market Segment,Parent Customer,Sale Origin,Sales by Parent,Suffix,Vistar Retail
0,34.99,Amazon SC,SO-00099997,34.99,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2473552.55,NaN,No
1,34.99,Amazon SC,SO-00099972,34.99,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2473552.55,NaN,No
2,34.99,Amazon SC,SO-00100026,34.99,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2473552.55,NaN,No
3,34.99,Amazon SC,SO-00100069,34.99,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2473552.55,NaN,No
4,34.99,Amazon SC,SO-00100085,34.99,2023-05-17,50ct Pouch - Variety Pk Choc Bites,Online,Amazon,Unleashed,2473552.55,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...
86980,2575.12,Vistar/ VSA Corporate,SO-00074694,2.58,2022-04-01,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1978810.10,K,Yes
86981,1383.46,Vistar/ VSA Corporate,SO-00079448,1.38,2022-07-18,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1978810.10,K,Yes
86982,122.62,Vistar/ VSA Corporate,SO-00074694,122.62,2022-04-01,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1978810.10,NaN,Yes
86983,1509.23,Vistar/ VSA Corporate,SO-00079448,1.51,2022-07-18,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,Alternate Retail,Vistar Retail,Unleashed,1978810.10,K,Yes


In [45]:
sales_23 = df_selection[df_selection['Invoice Date'].dt.year == 2023].Dollars.sum()
sales_22 = df_selection[df_selection['Invoice Date'].dt.year == 2022].Dollars.sum()

yoy_diff = round(int(sales_23-sales_22) / sales_22,2)
yoy_diff.round(2)

0.21

In [11]:
df_selection.columns

Index(['Dollars', 'Customer', 'Customer Order Number', 'Dollars Formatted',
       'Invoice Date', 'Item Full Description', 'Market Segment',
       'Parent Customer', 'Sale Origin', 'Sales by Parent', 'Suffix',
       'Vistar Retail'],
      dtype='object')

In [18]:
df_selection = df[
    (df['Invoice Date'].dt.year.isin(year)) & (df['Segment Description 2'].isin(segment))
    ][['Table',
       'Invoice Date',
       'Parent Customer',
       'Customer Name',
       'MFG #',
       'Item Full Description',
       'Qty Ordered',
       'Qty Received',
       'Dollars',
       'Segment Description 2']]

df_selection

,Table,Invoice Date,Parent Customer,Customer Name,MFG #,Item Full Description,Qty Ordered,Qty Received,Dollars,Segment Description 2
0,Dot,2022-01-03,J. POLEP,J. POLEP,00453U,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,8,8.0,1046.40,CONVENIENCE STORE
1,Dot,2022-01-03,J. POLEP,J. POLEP,00460U,CAFFEINATED CHOCOLATE BITES SINGLES PB CHOC,8,8.0,1046.40,CONVENIENCE STORE
2,Dot,2022-01-03,R.C. TAYLOR,R.C. TAYLOR,00105U,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,1,1.0,78.48,CONVENIENCE STORE
3,Dot,2022-01-03,Tropical,TROPICAL FOODS--TX,00150U,CAFFEINATED CHOCOLATE BITES SINGLES,2,2.0,261.60,ALTERNATE RETAIL
4,Dot,2022-01-03,Tropical,TROPICAL FOODS--TX,00453U,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,2,2.0,261.60,ALTERNATE RETAIL
...,...,...,...,...,...,...,...,...,...,...
16653,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00254U,AWAKE CHOCOLATE AWAKE CAFF DARK CHOC- 6X12PK M...,5,5.0,427.70,VENDING
16654,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00493U,AWAKE NO SUGAR ADDED 6 X 50CT BITES CHANGEMAKE...,3,3.0,423.00,VENDING
16655,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00494U,AWAKE NO SUGAR ADDED 6 X 50CT BITES CHANGEMAKE...,2,2.0,282.00,VENDING
16656,Dot,2023-12-08,Vistar,VISTAR-PHOENIX,00254U,AWAKE CHOCOLATE AWAKE CAFF DARK CHOC- 6X12PK M...,2,2.0,171.08,VENDING


In [11]:
px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'},
    width=800
).update_layout(showlegend=False)

In [ ]:
year = df['Invoice Date'].dt.year.unique()
segment = df['Segment Description 2'].unique()


df_selection = df[(df['Invoice Date'].dt.year.isin(year)) & (df['Segment Description 2'].isin(segment))]
# df_selection = df[(df['Invoice Date'].dt.year==2023) & (df['Segment Description 2'].isin(segment))]


seg_sales = df_selection.groupby('Segment Description 2',as_index=False)['Dollars'].sum()
fig_seg_sales = px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'}
)
fig_seg_sales.show()

In [ ]:
px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'}
)

In [30]:
df_selection.groupby(['Parent Customer','Segment Description 2'],as_index=False)['Dollars'].sum().set_index('Parent Customer').sort_values(by='Dollars',ascending=False)

,Segment Description 2,Dollars
Parent Customer,,
Vistar,VENDING,4.351449e+06
Vistar Retail,ALTERNATE RETAIL,1.941548e+06
Tropical,ALTERNATE RETAIL,1.256787e+06
J. POLEP,CONVENIENCE STORE,3.475965e+05
KELLI'S GIFT SHOP SUPPLIE,ALTERNATE RETAIL,3.039850e+05
...,...,...
C.M. TANNER GROCERY CO.,BROADLINE DISTRIBUTOR,2.154000e+01
TYLER C-STORE WHOLESALE,CONVENIENCE STORE,0.000000e+00
"F. MCCONNELL & SONS, INC.",ONLINE DISTRIBUTORS,0.000000e+00


In [37]:
parent_sales = df_selection.groupby(['Parent Customer','Segment Description 2'],as_index=False)['Dollars'].sum().set_index('Parent Customer').sort_values(by='Dollars',ascending=False)
fig_parent_sales = px.bar(
    parent_sales,
#     x=parent_sales.index,
    y='Dollars',
    color='Segment Description 2',
    title='<b>Sales by Parent</b>',
    template = 'plotly_white',
    labels={'Parent Customer':'',
            'Dollars':'<b>$USD</b>'}
)

In [40]:
fig_parent_sales.update_layout(showlegend=False).show()

In [42]:
import seaborn as sns
penguins = sns.load_dataset("penguins")
penguins

C:\Users\mikej\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning:

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2



,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female
